<a href="https://colab.research.google.com/github/paulandrea0/Autoencoder-Extracci-n-caracter-sticas/blob/main/Autoencoder_para_extraer_caracter%C3%ADsticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# Cargando el banco de datos de mi Drive

El set de datos ya está dividido en set_train y set_test

In [7]:
import pandas as pd
import numpy as np
df_train = pd.read_csv('/gdrive/MyDrive/Cardiacos/deteccion-anomalias-cardiacas-autoencoders-main/ECG5000_train.csv')
df_test = pd.read_csv('/gdrive/MyDrive/Cardiacos/deteccion-anomalias-cardiacas-autoencoders-main/ECG5000_test.csv')
print(df_train.shape)
print(df_test.shape)
X_train = np.array(df_train)
X_test = np.array(df_test)
df_train['0'].value_counts()

(7600, 141)
(1900, 141)


1    4427
2    2683
4     306
3     149
5      35
Name: 0, dtype: int64

# Autoencoder para extracción de características de las señales cardiacas

Normalización:

In [5]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

X_train_s = min_max_scaler.fit_transform(X_train)
print('Mínimo y máximo originales: {:.1f}, {:.1f}'.format(np.min(X_train), np.max(X_train)))      
print('Mínimo y máximo normalizado: {:.1f}, {:.1f}'.format(np.min(X_train_s), np.max(X_train_s))) 
X_test_s = min_max_scaler.transform(X_test)

Mínimo y máximo originales: -7.1, 7.4
Mínimo y máximo normalizado: 0.0, 1.0


In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras import Sequential
from keras.layers import Dropout  
from keras.callbacks import EarlyStopping

A continuación la configuración del autoencoder para reducir de 141 caracteres que tiene cada señal cardiaca para 20 caracteres. Escojo el valor de reducción de 141 a 20 de forma empírica (prueba y ajuste).

In [9]:
dim_entrada = X_train_s.shape[1]      
entrada = Input(shape=(dim_entrada,))
print(X_train_s.shape[1])             

# Encoder
x = Dense(80, activation='relu', )(entrada) # aqui de la entrada de 141 , ya lo reduce a 80
x = Dense(40, activation='relu')(x)
codificacion = Dense(20, activation='relu', name='bottleneck_layer')(x) # este es el cuello de botella, que será de aqui en adelante el set de datos de entrenamiento de forma codificada

# Decoder 
x = Dense(40, activation='relu')(codificacion)
x = Dense(80, activation='relu')(x)
decoded = Dense(141, activation='sigmoid')(x)

# autoencoder = entrada + decoder (que contiene el encoder)
autoencoder = Model(inputs=entrada, outputs=decoded)

autoencoder.summary()

141
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 141)]             0         
                                                                 
 dense (Dense)               (None, 80)                11360     
                                                                 
 dense_1 (Dense)             (None, 40)                3240      
                                                                 
 bottleneck_layer (Dense)    (None, 20)                820       
                                                                 
 dense_2 (Dense)             (None, 40)                840       
                                                                 
 dense_3 (Dense)             (None, 80)                3280      
                                                                 
 dense_4 (Dense)             (None, 141)               11

Ahora paso los datos de entrenamiento y prueba en el modelo de autoencoder que acabo de configurar para obtener una representación codificada (el cuello de botella):

In [10]:
bottleneck = Model(entrada, codificacion) 
test_codificado = bottleneck.predict(X_test_s)
train_codificado = bottleneck.predict(X_train_s)
print(test_codificado.shape)
print(train_codificado.shape)

238/238 [==============================] - 1s 4ms/step
(1900, 20)
(7600, 20)
